In [ ]:
# default_exp datasets

In [ ]:
#hide
%load_ext autoreload
%autoreload 2


# Dataset Classes

>Classes extending `torch.utils.data.Dataset` for the `torch.utils.data.DataLoader` objects to iterate over.

Two different classes are provided:

* `FaceClassificationDataset` to iterate over training/validation/test pictures, returning a tensor image and its corresponding label (index) in training/validation mode, and a tensor_image in test mode.

* `FaceClassificationDataset` to iterate over training/validation/test pictures, returning a tensor image and its label (index), in training mode, a pair of tensor images and their corresponding label, in validation mode, and a pair of tensor images in test mode.

To convert `.jpg` images to tensor, the Image module from `Pillow` is used.





In [ ]:
#export
# imports
import os
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision
from PIL import Image

In [ ]:
#export
class FaceClassificationDataset(Dataset):
    """
    """
    def __init__(self,
                 sample=None,
                 mode='train'):

        # Assertions to avoid wrong inputs
        assert mode in ['train', 'val', 'test']
        assert mode == 'test' and sample == None or \
            mode != 'test'
        if sample is not None:
            assert isinstance(sample, (list, np.ndarray))

        self.mode = mode

        # Directory setup
        if mode == 'train': 
            self.data_dir = './data/s1/train_data'
        elif mode == 'val':
            self.data_dir = './data/s1/val_data'
        else:
            self.data_dir = './data/s1/test_data'

        # Labels
        if (mode == 'train' or mode == 'val'):
            if sample is not None:
                sample = np.array(sample)
                assert sample.min() >= 0
                self.labels = np.array(sample)
                self.labels.sort(axis=0)
            else: 
               self.labels = [int(d) for d in os.listdir(self.data_dir)]
               self.labels = np.array(self.labels)
               self.labels.sort(axis=0)
        else:
            self.labels = os.listdir(self.data_dir)
            self.labels = np.array([int(f.split('.')[0]) for f in self.labels])
            self.labels.sort(axis=0)

        if mode != 'test':
            self.map_files = []
            for l in self.labels:
                temp_ls = [(l, f) for f in \
                    os.listdir(os.path.join(self.data_dir, str(l)))]
                temp_ls = [(t[0], t[1]) for t in temp_ls]
                self.map_files.append(temp_ls)

            self.map_files = [t for sl in self.map_files for t in sl]
            self.labels = [t[0] for t in self.map_files]
            self.X = [t[1] for t in self.map_files]  
        else:
            self.X = [str(s) + '.jpg' for s in self.labels]
            self.labels = ['' for s in self.labels]
   
    def __len__(self): return len(self.X)

    def __getitem__(self, idx):
        image_path = os.path.join(self.data_dir, 
                                  str(self.labels[idx]), 
                                  self.X[idx])
        image_tensor = Image.open(image_path)
        image_tensor = torchvision.transforms.ToTensor()(image_tensor)
        if self.mode == 'test': return image_tensor
        else: return image_tensor, self.labels[idx]

In [ ]:
#export
class FaceVerificationDataset(Dataset):
    """
    """
    def __init__(self,
                 sample=None,
                 mode='train'):

        # Assertions to avoid wrong inputs
        assert mode in ['val', 'test']
        assert mode == 'test' and sample == None or \
            mode != 'test'
        if sample is not None:
            assert isinstance(sample, (list, np.ndarray))

        self.mode = mode
        self.data_dir = './data/s2/'

        # Directory setup
        if mode == 'val': 
            self.pairs_file = \
                './data/s2/verification_pairs_val.txt'
        else:
            self.pairs_file = \
                './data/s2/verification_pairs_test.txt'

        with open(self.pairs_file) as f:
            self.pairs = [l.rstrip().split() for l in f]

        if sample is not None:
            sample = np.array(sample)
            sample.sort(axis=0)
            self.pairs = [self.pairs[i] for i in sample]            
   
    def __len__(self): return len(self.pairs)

    def __getitem__(self, idx):
        image_tensor_0 = Image.open(os.path.join(self.data_dir,self.pairs[idx][0]))
        image_tensor_1 = Image.open(os.path.join(self.data_dir,self.pairs[idx][1]))
        image_tensor_0 = torchvision.transforms.ToTensor()(image_tensor_0)
        image_tensor_1 = torchvision.transforms.ToTensor()(image_tensor_1)
        if self.mode == 'test': 
            return image_tensor_0, image_tensor_1
        else: 
            return image_tensor_0, image_tensor_1, int(self.pairs[idx][2])

## Tests

In [ ]:
import os

In [ ]:
print(os.getcwd())
os.chdir('..')

In [ ]:
print(os.getcwd())

In [ ]:
sample = np.array(range(3))
train_dataset = FaceClassificationDataset(sample, mode='train')
val_c_dataset = FaceClassificationDataset(sample, mode='val')
val_v_dataset = FaceVerificationDataset(sample, mode='val')

print(val_v_dataset.__getitem__(1))

(tensor([[[0.2588, 0.2471, 0.2510,  ..., 0.2824, 0.2902, 0.2941],
         [0.2627, 0.2510, 0.2549,  ..., 0.2863, 0.2941, 0.2980],
         [0.2588, 0.2431, 0.2471,  ..., 0.2941, 0.2980, 0.2980],
         ...,
         [0.0039, 0.0549, 0.0000,  ..., 0.0784, 0.0627, 0.0549],
         [0.0196, 0.0392, 0.0627,  ..., 0.0824, 0.0471, 0.0275],
         [0.0627, 0.0000, 0.1765,  ..., 0.0510, 0.0431, 0.0510]],

        [[0.2549, 0.2431, 0.2392,  ..., 0.2588, 0.2706, 0.2745],
         [0.2627, 0.2471, 0.2431,  ..., 0.2627, 0.2745, 0.2784],
         [0.2588, 0.2431, 0.2392,  ..., 0.2706, 0.2784, 0.2784],
         ...,
         [0.0157, 0.0667, 0.0000,  ..., 0.0863, 0.0706, 0.0627],
         [0.0275, 0.0471, 0.0706,  ..., 0.0902, 0.0588, 0.0392],
         [0.0706, 0.0000, 0.1843,  ..., 0.0627, 0.0549, 0.0588]],

        [[0.0667, 0.0627, 0.0706,  ..., 0.0627, 0.0549, 0.0549],
         [0.0667, 0.0588, 0.0667,  ..., 0.0588, 0.0549, 0.0510],
         [0.0549, 0.0471, 0.0510,  ..., 0.0667, 0.0588, 0